<a href="https://colab.research.google.com/github/clv07/ece251c/blob/main/rnn_fb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
# import torchaudio
from giant_fft_resample import giant_fft_resample
import rnn
import soundfile as sf


audio_path = 'audio/test_signal_input.wav'
model_path = 'BlackstarHT40_AmpHighGain.json'

os_ratio = [160, 147]       # [new_freq, orig_freq] e.g [160, 147] for SR conversion from 44.1kHz to 48kHz
method = 'apdl'             # sample rate conversion method
num_samples = -1            # option to truncate input audio


SNR = 38.290164947509766 dB


In [8]:
def computeSNR(audio_path, model_path, method, os_ratio):
    in_sig, base_sample_rate = sf.read(audio_path)
    in_sig = torch.from_numpy(in_sig).float().unsqueeze(0)

    # load models
    base_model = rnn.get_AudioRNN_from_json(model_path)
    model = rnn.get_SRIndieRNN(base_model=base_model, method=method)
    model.rec.os_factor = os_ratio[0] / os_ratio[1]

    with torch.no_grad():

        # oversample
        in_sig_os = giant_fft_resample(in_sig, orig_freq=os_ratio[1], new_freq=os_ratio[0])
        #  process
        out_sig_os, _ = model(in_sig_os)
        # downsample
        out_sig = giant_fft_resample(out_sig_os, orig_freq=os_ratio[0], new_freq=os_ratio[1])

        # target
        out_sig_target, _ = base_model(in_sig)

    # compute SNR compared to target
    diff = out_sig[..., :out_sig_target.shape[-1]].flatten() - out_sig_target.flatten()
    snr = out_sig_target.flatten().square().sum() / diff.square().sum()
    snr_dB = 10 * torch.log10(snr)
    return snr_dB


In [9]:
method = ['lidl', 'apdl','stn', 'cidl']
for m in method:
    snr_dB = computeSNR(audio_path, model_path, m, os_ratio)
    print(f'method = {m}, SNR = {snr_dB} dB')

method = lidl, SNR = 29.510499954223633 dB
method = apdl, SNR = 38.290164947509766 dB
method = stn, SNR = 28.271730422973633 dB
method = cidl, SNR = 45.586402893066406 dB
